# 2-Body Problem

* Last revised 25-Apr-2022 by Matt Shmukler (shmukler.2@osu.edu).

We consider energy plots and orbital solutions in polar coordinates for the general potential energy

$\begin{align}
   U(r) = -\frac{Gm_1m_2}{r} \\
\end{align}$
where $r$ is the distance between the two particles.
We shall solve the 2-body problem with the lagrangian method.
We have our kinetic energy 
$$T=\frac{1}{2}m_1v_1^2+\frac{1}{2}m_2v_2^2=\frac{1}{2}m_1(\dot x_1^2+\dot y_1^2)+\frac{1}{2}m_2(\dot x_2^2+\dot y_2^2)$$
and our potential energy
$$U=-\frac{Gm_1m_2}{\sqrt{(x_1-x_2)^2+(y_1-y_2)^2}}$$
This gives us a lagrangian 
$$\mathcal{L}=\frac{1}{2}m_1(\dot x_1^2+\dot y_1^2)+\frac{1}{2}m_2(\dot x_2^2+\dot y_2^2)+\frac{Gm_1m_2}{\sqrt{(x_1-x_2)^2+(y_1-y_2)^2}}$$
We can now solve the euler lagrange equations
$$\frac{d\mathcal{L}}{dx_i}=\frac{d}{dt}\frac{d\mathcal{L}}{d\dot x_i}$$
where $x_i=x_1,x_2,y_1,y_2$

This gives us four equations which we shall solve numerically:

$$\ddot x_1=-\frac{Gm_2(x_1-x_2)}{(\sqrt{(x_1-x_2)^2+(y_1-y_2)^2})^3}$$

$$\ddot x_2=\frac{Gm_1(x_1-x_2)}{(\sqrt{(x_1-x_2)^2+(y_1-y_2)^2})^3}$$

$$\ddot y_1=-\frac{Gm_2(y_1-y_2)}{(\sqrt{(x_1-x_2)^2+(y_1-y_2)^2})^3}$$

$$\ddot y_2=\frac{Gm_1(y_1-y_2)}{(\sqrt{(x_1-x_2)^2+(y_1-y_2)^2})^3}$$

We now implement these equations as a class. There will be two different solving methods available: the SciPy solve_ivp method and the Leapfrog method.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

In [ ]:
# Change the common font size
font_size = 14
plt.rcParams.update({'font.size': font_size})

In [ ]:
class GravityOrbit():
    """
    This class implements lagranges equations for two masses orbiting each other 
    with a gravitational interaction.
    
    Parameters
    ----------
    m_1:float
    mass 1
    m_2:float
    mass 2
    G:float
    Gravitational constant
    
    Methods
    -------
    dy_dt(t,y)
        Returns the RHS of differential equation in y for a given time t and value of y.
    """
    
    def __init__(self, m_1=1., m_2=1., G=1.):
        #Initialize members of class
        self.m_1 = m_1
        self.m_2 = m_2
        self.G = G
        
    def dy_dt(self, t, y):
        """
        This function returns the right-hand side of the diffeq: 
        [dy/dt d^2y/dt^2]
        
        Parameters
        ----------
        t : float
            time 
        y : float
            8-component vector with y[0] = x1(t), y[1] = x1_dot,
                                    y[2] = x2(t), y[3] = x2_dot,
                                    y[4] = y1(t), y[5] = y1_dot,
                                    y[6] = y2(t), y[7] = y2_dot,
            
        """
        r=np.sqrt((y[0]-y[2])**2 + (y[4]-y[6])**2)
        return [y[1], -self.G*self.m_2*(y[0]-y[2])/r**3,\
                y[3], self.G*self.m_1*(y[0]-y[2])/r**3,\
                y[5], -self.G*self.m_2*(y[4]-y[6])/r**3,\
                y[7], self.G*self.m_1*(y[4]-y[6])/r**3 ]
    
    
    def solve_ode(self, t_pts, y_0, 
                  abserr=1.0e-8, relerr=1.0e-8):
        """
        Solve the ODE given initial conditions.
        Specify smaller abserr and relerr to get more precision.
        """
        solution = solve_ivp(self.dy_dt, (t_pts[0], t_pts[-1]), 
                             y_0, t_eval=t_pts, 
                             atol=abserr, rtol=relerr)
        x1, x1_dot, x2, x2_dot, y1, y1_dot, y2, y2_dot = solution.y
        return x1, x1_dot, x2, x2_dot, y1, y1_dot, y2, y2_dot
    
    def solve_frog(self, t_pts, y_0):
        """
        Solve the ODE given initial conditions using the leapfrog method, which conserves energy.
        """
        delta_t=t_pts[1]-t_pts[0]
        
        #Load in initial conditions
        x1_0, x1_dot_0, x2_0, x2_dot_0, y1_0, y1_dot_0, y2_0, y2_dot_0 = y_0
        
        #Initialize arrays
        num_t_pts=len(t_pts)
        
        x1=np.zeros(num_t_pts)
        x1_dot=np.zeros(num_t_pts)
        x1_dot_half=np.zeros(num_t_pts)
        
        x2=np.zeros(num_t_pts)
        x2_dot=np.zeros(num_t_pts)
        x2_dot_half=np.zeros(num_t_pts)
        
        y1=np.zeros(num_t_pts)
        y1_dot=np.zeros(num_t_pts)
        y1_dot_half=np.zeros(num_t_pts)
        
        y2=np.zeros(num_t_pts)
        y2_dot=np.zeros(num_t_pts)
        y2_dot_half=np.zeros(num_t_pts)
        
        #Initial conditions
        x1[0] = x1_0
        x1_dot[0] = x1_dot_0  
        x2[0] = x2_0
        x2_dot[0] = x2_dot_0
        y1[0] = y1_0
        y1_dot[0] = y1_dot_0
        y2[0] = y2_0
        y2_dot[0] = y2_dot_0
        
        #Implement leapfrog method
        for i in np.arange(num_t_pts-1):
            t = t_pts[i]
            y = [x1[i], x1_dot[i], x2[i], x2_dot[i], y1[i], y1_dot[i], y2[i], y2_dot[i]]
            derivs = self.dy_dt(t,y)
            
            x1_dot_half[i]=x1_dot[i]+derivs[1]*delta_t/2.
            x1[i+1]=x1[i]+x1_dot_half[i]*delta_t
            
            x2_dot_half[i]=x2_dot[i]+derivs[3]*delta_t/2.
            x2[i+1]=x2[i]+x2_dot_half[i]*delta_t
            
            y1_dot_half[i]=y1_dot[i]+derivs[5]*delta_t/2.
            y1[i+1]=y1[i]+y1_dot_half[i]*delta_t
            
            y2_dot_half[i]=y2_dot[i]+derivs[7]*delta_t/2.
            y2[i+1]=y2[i]+y2_dot_half[i]*delta_t
            
            y=[x1[i+1], x1_dot[i], x2[i+1], x2_dot[i], y1[i+1], y1_dot[i], y2[i+1], y2_dot[i]]
            derivs = self.dy_dt(t,y)
            
            x1_dot[i+1]=x1_dot_half[i]+derivs[1]*delta_t/2.
            x2_dot[i+1]=x2_dot_half[i]+derivs[3]*delta_t/2.
            y1_dot[i+1]=y1_dot_half[i]+derivs[5]*delta_t/2.
            y2_dot[i+1]=y2_dot_half[i]+derivs[7]*delta_t/2.
        
        return x1, x1_dot, x2, x2_dot, y1, y1_dot, y2, y2_dot
    
    def energy(self,t_pts,x1, x1_dot, x2, x2_dot, y1, y1_dot, y2, y2_dot):
        """
        Returns the energy of the two-body system at a given time.
        """
        return 0.5*self.m_1*(x1_dot**2 + y1_dot**2)+0.5*self.m_2*(x2_dot**2 + y2_dot**2)\
                -self.G*self.m_1*self.m_2/(np.sqrt((x1-x2)**2 + (y1-y2)**2))

In [ ]:
def plot_y_vs_x(x, y, axis_labels=None, label=None, title=None, 
                color=None, linestyle=None, semilogy=False, loglog=False,
                ax=None):
    """
    Generic plotting function: return a figure axis with a plot of y vs. x,
    with line color and style, title, axis labels, and line label
    """
    if ax is None:        # if the axis object doesn't exist, make one
        ax = plt.gca()

    if (semilogy):
        line, = ax.semilogy(x, y, label=label, 
                            color=color, linestyle=linestyle)
    elif (loglog):
        line, = ax.loglog(x, y, label=label, 
                          color=color, linestyle=linestyle)
    else:
        line, = ax.plot(x, y, label=label, 
                    color=color, linestyle=linestyle)

    if label is not None:    # if a label if passed, show the legend
        ax.legend()
    if title is not None:    # set a title if one if passed
        ax.set_title(title)
    if axis_labels is not None:  # set x-axis and y-axis labels if passed  
        ax.set_xlabel(axis_labels[0])
        ax.set_ylabel(axis_labels[1])

    return ax, line


def start_stop_indices(t_pts, plot_start, plot_stop):
    start_index = (np.fabs(t_pts-plot_start)).argmin()  # index in t_pts array 
    stop_index = (np.fabs(t_pts-plot_stop)).argmin()  # index in t_pts array 
    return start_index, stop_index

# Simple Orbit Case

In [ ]:
#Labels for cartesian coordinates
cartesian_label=(r'$x$',r'$y$')

#Common plotting time (generate the full time then use slices)
t_start=0.
t_end=500.
delta_t=0.01
t_pts=np.arange(t_start,t_end+delta_t,delta_t)

#Parameter Values:
G=1.0
m_1=1.
m_2=5.

#Create instance of our orbit
orbit0=GravityOrbit(m_1, m_2, G)

#Initial Conditions such that Center of Mass is at origin and does not move
x1_0, x1_dot_0, y1_0, y1_dot_0 = 1., -1., 1., 1.
x2_0, x2_dot_0, y2_0, y2_dot_0 = -(m_1/m_2)*x1_0, -(m_1/m_2)*x1_dot_0, -(m_1/m_2)*y1_0, -(m_1/m_2)*y1_dot_0

y_0=[x1_0, x1_dot_0, x2_0, x2_dot_0, y1_0, y1_dot_0, y2_0, y2_dot_0]

#Solve system
x1s, x1_dots, x2s, x2_dots, y1s, y1_dots, y2s, y2_dots = orbit0.solve_ode(t_pts,y_0)

#Plot Results
fig1 = plt.figure(figsize=(5,5))
ax1 = fig1.add_subplot(1,1,1)
ax1.plot(x1s, y1s, color='blue', label=r'$m_1$')
ax1.plot(x2s, y2s, color='red', label=r'$m_2$')
ax1.set_title('Gravitational Orbit')
fig1.tight_layout()
ax1.legend()
ax1.set_aspect(1)

fig1.savefig('Gravitation_orbit_0.png')

# Special Case Orbit

In [ ]:
#Labels for cartesian coordinates
cartesian_label=(r'$x$',r'$y$')

#Common plotting time (generate the full time then use slices)
t_start=0.
t_end=500.
delta_t=0.01
t_pts=np.arange(t_start,t_end+delta_t,delta_t)

#Parameter Values: Make one mass significantly heavier than the other
G=0.005
m_1=1.
m_2=500.

#Create instance of our orbit
orbit1=GravityOrbit(m_1, m_2, G)

#Initial Conditions such that Center of Mass is at origin and does not move
x1_0, x1_dot_0, y1_0, y1_dot_0 = 1., -1., 1., 1.
x2_0, x2_dot_0, y2_0, y2_dot_0 = -(m_1/m_2)*x1_0, -(m_1/m_2)*x1_dot_0, -(m_1/m_2)*y1_0, -(m_1/m_2)*y1_dot_0

y_0=[x1_0, x1_dot_0, x2_0, x2_dot_0, y1_0, y1_dot_0, y2_0, y2_dot_0]

#Solve system
x1p, x1_dotp, x2p, x2_dotp, y1p, y1_dotp, y2p, y2_dotp = orbit1.solve_ode(t_pts,y_0)

#Plot Results
fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(1,1,1)
ax.plot(x1p, y1p, color='blue', label=r'$m_1$')
ax.plot(x2p, y2p, color='red', label=r'$m_2$')
ax.set_title('Gravitational Orbit')
fig.tight_layout()
ax.legend()
ax.set_aspect(1)

fig.savefig('Gravitation_orbit_1.png')


In this special case, we have $m_2\gg m_1$ and we are in a frame where the center of mass of the system is at the origin and is not moving. For our orbits considered in class, these same assumptions were made. We found that the heavier mass was essentially stationary and the problem reduced down to solving for the motion of the lighter mass. We could simply solve for the radial coordinate and angular coordinate of the lighter mass and we were done. In this special case, I show that $m_2$ is essentially stationary at the origin (center of mass) while the lighter mass orbits around. With some algebraic manipulation, we can reduce the system down to a one body problem. Thus we have shown that if one mass is significantly heavier than the other and we are in the center of mass frame (COM stationary), the orbits reduce to the ones we covered in class.

# Simple Orbit with Leapfrog Method

In [ ]:
#Labels for cartesian coordinates
cartesian_label=(r'$x$',r'$y$')

#Common plotting time (generate the full time then use slices)
t_start=0.
t_end=500.
delta_t=0.01
t_pts=np.arange(t_start,t_end+delta_t,delta_t)

#Parameter Values: 
G=1.0
m_1=1.
m_2=5.

#Create instance of our orbit
orbit2=GravityOrbit(m_1, m_2, G)

#Initial Conditions such that Center of Mass is at origin and does not move
x1_0, x1_dot_0, y1_0, y1_dot_0 = 1., -1., 1., 1.
x2_0, x2_dot_0, y2_0, y2_dot_0 = -(m_1/m_2)*x1_0, -(m_1/m_2)*x1_dot_0, -(m_1/m_2)*y1_0, -(m_1/m_2)*y1_dot_0

y_0=[x1_0, x1_dot_0, x2_0, x2_dot_0, y1_0, y1_dot_0, y2_0, y2_dot_0]

#Solve system using Leapfrog method
x1l, x1_dotl, x2l, x2_dotl, y1l, y1_dotl, y2l, y2_dotl = orbit2.solve_frog(t_pts,y_0)

#Plot Results
fig2 = plt.figure(figsize=(5,5))
ax2 = fig2.add_subplot(1,1,1)
ax2.plot(x1l, y1l, color='blue', label=r'$m_1$')
ax2.plot(x2l, y2l, color='red', label=r'$m_2$')
ax2.set_title('Gravitational Orbit')
fig2.tight_layout()
ax2.legend()
ax2.set_aspect(1)

fig2.savefig('Gravitation_orbit_leapfrog.png')

# Compare energy over time between Leapfrog and SciPy ODE Solver

In [ ]:
#Energy calculations for SciPy ODE Solver
E_tot_pts = orbit0.energy(t_pts, x1s, x1_dots, x2s, x2_dots, y1s, y1_dots, y2s, y2_dots)
E_tot_0 = E_tot_pts[0]
E_tot_rel_pts = np.abs((E_tot_pts - E_tot_0)/E_tot_0)

#Energy calculations for Leapfrog Solution
E_tot_pts_LF = orbit2.energy(t_pts, x1l, x1_dotl, x2l, x2_dotl, y1l, y1_dotl, y2l, y2_dotl)
E_tot_0_LF = E_tot_pts_LF[0]
E_tot_rel_pts_LF = np.abs((E_tot_pts_LF - E_tot_0_LF)/E_tot_0_LF)

start, stop = start_stop_indices(t_pts, t_start, 10.)

#Plot energy over time
fig_5 = plt.figure(figsize=(12,6))
ax_5a = fig_5.add_subplot(1,2,1)
ax_5a.semilogy(t_pts[start:stop], E_tot_rel_pts[start:stop], 
               color='green', label=r'$\Delta E(t)$ SciPy ODE')
ax_5a.semilogy(t_pts[start:stop], E_tot_rel_pts_LF[start:stop], 
               color='red', label=r'$\Delta E(t)$ Leapfrog')
ax_5a.set_ylim(1.e-10, 1.e-2)    # (1.e-12, 5)
ax_5a.set_xlabel(r'$t$')
ax_5a.set_ylabel(r'Energy')
ax_5a.set_title('Change in energy with time')
ax_5a.legend()
ax_5b = fig_5.add_subplot(1,2,2)
ax_5b.semilogy(t_pts, E_tot_rel_pts, 
               color='green', label=r'$\Delta E(t)$ SciPy ODE')
ax_5b.semilogy(t_pts, E_tot_rel_pts_LF, 
               color='red', label=r'$\Delta E(t)$ Leapfrog')
ax_5b.set_ylim(1.e-10, 1.e-2)    # (1.e-12, 5)
ax_5b.set_xlabel(r'$t$')
ax_5b.set_ylabel(r'Energy')
ax_5b.set_title('Change in energy with time')
ax_5b.legend()

fig_5.tight_layout()
fig_5.savefig('Leapfrog_energy_test_1.png', dpi=200, bbox_inches='tight')


From the two plots above, we can see that the energy for the leapfrog method is conserved and will never move too far away from its original value (looks like energy at any time varies by at most 10e-4 from the starting value). On the other hand, we see that the change in energy for the SciPy ODE solver is slowly increasing over time. If we were to run this simulation for a longer time, this change in energy would keep increasing. Thus, energy is not conserved when solving with the SciPy ODE solver. This becomes an issue when running the simulation for long periods of time, as energy will slowly "seep" out of the system. 